In [1]:
!pip list | grep neuron

libneuronxla                  2.2.1630.0
neuronx-cc                    2.17.194.0+d312836f
neuronx-distributed           0.11.0
neuronx-distributed-inference 0.2.0
torch-neuronx                 2.5.1.2.6.0


In [ ]:
import torch
from transformers import AutoTokenizer, GenerationConfig
from neuronx_distributed_inference.models.config import NeuronConfig, OnDeviceSamplingConfig
from neuronx_distributed_inference.utils.hf_adapter import HuggingFaceGenerationAdapter, load_pretrained_config

In [2]:
model_path = "/home/ubuntu/model_hf_qwen/qwen/"
traced_model_path = "/home/ubuntu/traced_model_qwen/qwen/"

In [ ]:
from huggingface_hub import snapshot_download

snapshot_download("Qwen/Qwen3-8B", local_dir=model_path)

In [ ]:
from modeling_qwen import Qwen3InferenceConfig, NeuronQwen3ForCausalLM

def run_qwen3_compile():
    # Initialize configs and tokenizer.
    tokenizer = AutoTokenizer.from_pretrained(model_path, padding_side="right")
    tokenizer.pad_token = tokenizer.eos_token

    generation_config = GenerationConfig.from_pretrained(model_path)
    generation_config_kwargs = {
        "do_sample": True,
        "top_k": 1,
        "pad_token_id": tokenizer.pad_token_id,
    }
    generation_config.update(**generation_config_kwargs)
 
    neuron_config = NeuronConfig(
        tp_degree=8,
        batch_size=1,
        max_context_length=128,
        seq_len=256,
        on_device_sampling_config=OnDeviceSamplingConfig(top_k=5),
        enable_bucketing=True,
        context_encoding_buckets=[128],
        token_generation_buckets=[256],
        flash_decoding_enabled=False,
        torch_dtype=torch.bfloat16,
        fused_qkv=False,
        attn_kernel_enabled=True,
        attn_cls="NeuronQwen3Attention"
    )
    config = Qwen3InferenceConfig(
        neuron_config,
        load_config=load_pretrained_config(model_path),
    )
    
    # Compile and save model.
    print("\nCompiling and saving model...")
    model = NeuronQwen3ForCausalLM(model_path, config)
    model.compile(traced_model_path)
    tokenizer.save_pretrained(traced_model_path)

In [ ]:
run_qwen3_compile()

In [ ]:
from modeling_qwen import Qwen3InferenceConfig, NeuronQwen3ForCausalLM

model = NeuronQwen3ForCausalLM(traced_model_path)
model.load(traced_model_path)

In [ ]:
config = model.get_config_cls()
config.get_neuron_config_cls()

In [ ]:
model.config.num_attention_heads

In [ ]:
model.config.num_key_value_heads

In [ ]:
model.config.hidden_size

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(traced_model_path)
tokenizer.pad_token = tokenizer.eos_token
generation_config = GenerationConfig.from_pretrained(model_path)
generation_config_kwargs = {
    "do_sample": False,
    "temperature": 0.9,
    "top_k": 5,
    "pad_token_id": tokenizer.pad_token_id,
}
generation_config.update(**generation_config_kwargs)
generation_model = HuggingFaceGenerationAdapter(model)
messages = [{'role': 'user', 'content': "What's your name?"}]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=False # Switches between thinking and non-thinking modes. Default is True.
)
inputs = tokenizer([text], return_tensors="pt")

In [ ]:
print("\nGenerating outputs...")
outputs = generation_model.generate(
    **inputs,
    max_new_tokens=512
)

In [21]:
output_ids = outputs[0][len(inputs.input_ids[0]):].tolist() 

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip("\n")
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

thinking content: 
content: My name is Qwen, and I'm a large language model developed by Alibaba Cloud. How can I assist you today?


In [13]:
model.reset()

# Run Benchmarks

In [ ]:
dir = '/opt/aws_neuronx_venv_pytorch_2_5_nxd_inference/lib/python3.10/site-packages/neuronx_distributed_inference/'
!cp modeling_qwen.py {dir}

In [1]:
!inference_demo \
    --model-type qwen3 \
    --task-type causal-lm \
    run \
    --model-path /home/ubuntu/model_hf_qwen/qwen/ \
    --compiled-model-path /home/ubuntu/traced_model_qwen/qwen/logit \
    --torch-dtype bfloat16 \
    --tp-degree 8 \
    --batch-size 1 \
    --max-context-length 16 \
    --seq-len 32 \
    --enable-bucketing \
    --pad-token-id 151645 \
    --prompt "To be, or not to be" \
    --check-accuracy-mode logit-matching \
    --benchmark

/opt/aws_neuronx_venv_pytorch_2_5_nxd_inference/lib/python3.10/site-packages/neuronx_distributed/modules/moe/expert_mlps.py:11: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from neuronx_distributed.modules.moe.blockwise import (
/opt/aws_neuronx_venv_pytorch_2_5_nxd_inference/lib/python3.10/site-packages/neuronx_distributed/modules/moe/expert_mlps.py:11: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from neuronx_distributed.modules.moe.blockwise import (
/opt/aws_neuronx_venv_pytorch_2_5_nxd_inference/lib/python3.10/site-packages/neuronx_distributed/modules/moe/expert_mlps.py:11: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit instead.
  from neuronx_distributed.modules.moe.blockwise import (
/opt/aws_neuronx_venv_pytorch_2_5_nxd_inference/lib/python3.10/site-packages/neuronx_distributed_inference/modules/attention/utils.py:14: DeprecationWarning: torch_neuronx.nki_jit is deprecated, use nki.jit i